In [1]:
import pyspark
pyspark.__version__

'3.5.1'

In [2]:
from pyspark.sql import SparkSession

In [3]:
# Create SparkSession object
spark = SparkSession.builder.master("local[1]").appName('test').getOrCreate()

24/08/06 07:26:18 WARN Utils: Your hostname, codespaces-648539 resolves to a loopback address: 127.0.0.1; using 10.0.3.203 instead (on interface eth0)
24/08/06 07:26:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/06 07:26:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [31]:
# Terminate the cluster
spark.stop()

In [5]:
spark.version

'3.5.1'

### Read a csv file

In [4]:
# Read data from CSV file
df = spark.read.csv('flights.csv',
                         sep=',',
                         header=True,
                         inferSchema=True,
                         nullValue='NA')

# Get number of records
print("The data contain %d records." % df.count())

# View the first five records
display(df.show(5))

# Check column data types
print(df.dtypes)

The data contain 15001 records.
+---+---+---+-------+------+---+----+------+--------+-----+
|mon|dom|dow|carrier|flight|org|mile|depart|duration|delay|
+---+---+---+-------+------+---+----+------+--------+-----+
| 10| 10|  1|     OO|  5836|ORD| 157|  8.18|      51|   27|
|  1|  4|  1|     OO|  5866|ORD| 466|  15.5|     102| NULL|
| 11| 22|  1|     OO|  6016|ORD| 738|  7.17|     127|  -19|
|  2| 14|  5|     B6|   199|JFK|2248| 21.17|     365|   60|
|  5| 25|  3|     WN|  1675|SJC| 386| 12.92|      85|   22|
+---+---+---+-------+------+---+----+------+--------+-----+
only showing top 5 rows



None

[('mon', 'int'), ('dom', 'int'), ('dow', 'int'), ('carrier', 'string'), ('flight', 'int'), ('org', 'string'), ('mile', 'int'), ('depart', 'double'), ('duration', 'int'), ('delay', 'int')]


24/08/06 07:26:36 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [30]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType
# from pyspark.sql.types import *

# Specify column names and types
schema = StructType([
    StructField("mon", FloatType()),
    StructField("dom", IntegerType()),
    StructField("dow", IntegerType()),
    StructField("carrier", StringType()),
    StructField("depart", IntegerType()),
    StructField("flight", StringType())
])

# Load data from a delimited file
df = spark.read.csv('flights.csv', sep=',', header=True, schema=schema)

# Print schema of DataFrame
display(df.printSchema())

# View the first five records
display(df.show(5))

root
 |-- mon: float (nullable = true)
 |-- dom: integer (nullable = true)
 |-- dow: integer (nullable = true)
 |-- carrier: string (nullable = true)
 |-- depart: integer (nullable = true)
 |-- flight: string (nullable = true)



None

+----+---+---+-------+------+------+
| mon|dom|dow|carrier|depart|flight|
+----+---+---+-------+------+------+
|10.0| 10|  1|     OO|  5836|   ORD|
| 1.0|  4|  1|     OO|  5866|   ORD|
|11.0| 22|  1|     OO|  6016|   ORD|
| 2.0| 14|  5|     B6|   199|   JFK|
| 5.0| 25|  3|     WN|  1675|   SJC|
+----+---+---+-------+------+------+
only showing top 5 rows



24/08/06 06:20:56 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 10, schema size: 6
CSV file: file:///workspaces/data_science_materials/pyspark/flights.csv


None

### Drop Column & Remove Nulls

In [6]:
# Remove the 'flight' column
flights_drop_column = df.drop('flight')
print(flights_drop_column.count())

# Number of records with missing 'delay' values
print(flights_drop_column.filter('delay IS NULL').count())

# Remove records with missing 'delay' values
flights_valid_delay = flights_drop_column.filter('delay IS NOT NULL')
print(flights_valid_delay.count())

# Remove records with missing values in any column and get the number of remaining rows
flights_none_missing = flights_valid_delay.dropna()
print(flights_none_missing.count())

15001
915
14086
14086
